In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls /content/drive/MyDrive/AffectNet/

train_set.tar  val_set.tar


In [7]:
!mkdir /content/AffectNet

mkdir: cannot create directory ‘/content/AffectNet’: File exists


In [8]:
!cp -r /content/drive/MyDrive/AffectNet/ /content/AffectNet/

In [9]:
!tar -xvf /content/AffectNet/AffectNet/train_set.tar -C /content/AffectNet/AffectNet/

Streaming output truncated to the last 5000 lines.
train_set/annotations/26361_lnd.npy
train_set/annotations/313969_aro.npy
train_set/annotations/286825_exp.npy
train_set/annotations/392215_aro.npy
train_set/annotations/330087_lnd.npy
train_set/annotations/110001_val.npy
train_set/annotations/371482_aro.npy
train_set/annotations/27940_aro.npy
train_set/annotations/35729_exp.npy
train_set/annotations/156731_aro.npy
train_set/annotations/145081_aro.npy
train_set/annotations/93043_exp.npy
train_set/annotations/279607_lnd.npy
train_set/annotations/48472_lnd.npy
train_set/annotations/206512_exp.npy
train_set/annotations/84264_aro.npy
train_set/annotations/187769_lnd.npy
train_set/annotations/300707_exp.npy
train_set/annotations/123766_exp.npy
train_set/annotations/173216_aro.npy
train_set/annotations/262096_lnd.npy
train_set/annotations/194946_aro.npy
train_set/annotations/411301_aro.npy
train_set/annotations/43103_aro.npy
train_set/annotations/288373_aro.npy
train_set/annotations/9784_exp.

In [82]:
!tar -xvf /content/AffectNet/AffectNet/val_set.tar -C /content/AffectNet/AffectNet/

Streaming output truncated to the last 5000 lines.
val_set/annotations/678_lnd.npy
val_set/annotations/1526_lnd.npy
val_set/annotations/5420_aro.npy
val_set/annotations/4628_exp.npy
val_set/annotations/5254_lnd.npy
val_set/annotations/4769_aro.npy
val_set/annotations/1994_lnd.npy
val_set/annotations/4840_lnd.npy
val_set/annotations/3546_val.npy
val_set/annotations/2606_lnd.npy
val_set/annotations/1282_val.npy
val_set/annotations/948_lnd.npy
val_set/annotations/2672_lnd.npy
val_set/annotations/1599_aro.npy
val_set/annotations/2177_exp.npy
val_set/annotations/1830_val.npy
val_set/annotations/4676_aro.npy
val_set/annotations/3228_aro.npy
val_set/annotations/5196_aro.npy
val_set/annotations/714_val.npy
val_set/annotations/2296_exp.npy
val_set/annotations/934_lnd.npy
val_set/annotations/1511_val.npy
val_set/annotations/2923_exp.npy
val_set/annotations/4198_aro.npy
val_set/annotations/886_lnd.npy
val_set/annotations/1924_exp.npy
val_set/annotations/1462_lnd.npy
val_set/annotations/812_exp.np

In [10]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.5/105.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 48.3 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117029 sha256=a18d9d478470aa8a008ba70b3b7a83d6b77a8e35f1614738a51fb8553834ed65
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [31]:
from PIL import Image
from torchvision.io import read_image
import cv2
import numpy as np
from tqdm.auto import tqdm
from deepface.modules import modeling, preprocessing
from deepface.extendedmodels import Emotion
import torch
from torchvision import transforms
from torchvision.transforms.functional import to_pil_image
import numpy as np
import cv2
from PIL import Image
import tensorflow as tf
import os
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import precision_recall_fscore_support

In [14]:
if not os.path.exists('/root/.deepface/weights'):
    os.mkdir('/root/.deepface')
    os.mkdir('/root/.deepface/weights')
emotion_model = modeling.build_model("Emotion").model

24-07-27 12:00:33 - facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:00<00:00, 23.1MB/s]


In [15]:
TRAIN_IMG_PATH =  "/content/AffectNet/AffectNet/train_set/images"
TRAIN_LABELS_PATH =  "/content/AffectNet/AffectNet/train_set/annotations"
VAL_IMG_PATH =  "/content/AffectNet/AffectNet/val_set/images"
VAL_LABELS_PATH =  "/content/AffectNet/AffectNet/val_set/annotations"

In [62]:

class ResizeAndCombineClasses(object):
    def __init__(self, size):
        self.size = size  # Set the target size for resizing
#         self.resize = transforms.Resize(self.size)  # Initialize the resize transformation

    def __call__(self, image):

        # Preprocessing
        image = np.array(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (48, 48))
        image = np.expand_dims(image, axis=0)

#         # Convert to tensor (optional here depending on subsequent use)
        # image = transforms.ToTensor()(image)

        return image


In [63]:
class CustomAffectNetDataset(Dataset):
    def __init__(self, annotations_dir, img_dir, transform=None, target_transform=None):
        # store the image files in sorted order
        self.img_dir = img_dir
        self.label_dir = annotations_dir
        self.img_files = sorted(os.listdir(img_dir))
        # self.img_label_files = sorted([file for file in os.listdir(annotations_dir) if 'exp' in file])
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_files[idx])
        image = read_image(img_path)
        label_path = os.path.join(self.label_dir, f"{self.img_files[idx].split('.')[0]}_exp.npy")
        label = int(np.load(label_path))
        if self.transform:
            image = self.transform(to_pil_image(image))
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [64]:
training_data = CustomAffectNetDataset(TRAIN_LABELS_PATH, TRAIN_IMG_PATH, ResizeAndCombineClasses(48))

In [76]:
train_dataloader = DataLoader(training_data, batch_size=128, shuffle=True)

In [77]:
label_mapping = {label.lower(): i for i, label in enumerate(["Neutral", "Happy", "Sad", "Surprise", "Fear", "Disgust", "Angry"])}
print(label_mapping)

{'neutral': 0, 'happy': 1, 'sad': 2, 'surprise': 3, 'fear': 4, 'disgust': 5, 'angry': 6}


In [78]:
Emotion.labels

['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

In [79]:
correct_emotions = total = 0
all_emotion_preds = []
true_emotion_preds = []
for images, labels in tqdm(train_dataloader):
#     import pdb; pdb.set_trace()
    # Ignore contempt since deepface can't handle it
    label_to_ignore = 7
    relevance_mask = labels != label_to_ignore
    images = images[relevance_mask]
    labels = labels[relevance_mask]
    emotion_preds = emotion_model(np.array(images).squeeze())
    emotion_preds = np.argmax(emotion_preds, axis=1)
    emotion_preds = [label_mapping[Emotion.labels[pred]] for pred in emotion_preds]
    correct_emotions += np.sum(emotion_preds == np.array(labels))
    all_emotion_preds += emotion_preds
    true_emotion_preds += labels
    total += len(labels)

  0%|          | 0/2248 [00:00<?, ?it/s]

In [83]:
# Calculate F-score, precision, recall - Race - Training set
precision, recall, f_score_weighted, _ = precision_recall_fscore_support(true_emotion_preds, all_emotion_preds, average='weighted')
_, _, f_score_macro, _ = precision_recall_fscore_support(true_emotion_preds, all_emotion_preds, average='macro')
_, _, f_score_micro, _ = precision_recall_fscore_support(true_emotion_preds, all_emotion_preds, average='micro')
train_acc = np.sum(np.array(all_emotion_preds) == np.array(true_emotion_preds)) / total
print(f"Training set metrics - Emotion (Deepface) \n" + "="*40)
print(f"Accuracy: {train_acc:.4f} Precision: {precision:.4f}, Recall: {recall:.4f}, F-Score(Weighted): {f_score_weighted:.4f}, F-Score(Micro): {f_score_micro:.4f}, F-Score(Macro): {f_score_macro:.4f}")

Training set metrics - Emotion (Deepface) 
Accuracy: 0.5436 Precision: 0.5754, Recall: 0.5436, F-Score(Weighted): 0.5553, F-Score(Micro): 0.5436, F-Score(Macro): 0.3014


In [88]:
validation_data = CustomAffectNetDataset(VAL_LABELS_PATH, VAL_IMG_PATH, ResizeAndCombineClasses(48))

In [89]:
val_dataloader = DataLoader(validation_data, batch_size=128, shuffle=True)

In [90]:
correct_emotions = total = 0
all_emotion_preds = []
true_emotion_preds = []
for images, labels in tqdm(val_dataloader):
#     import pdb; pdb.set_trace()
    # Ignore contempt since deepface can't handle it
    label_to_ignore = 7
    relevance_mask = labels != label_to_ignore
    images = images[relevance_mask]
    labels = labels[relevance_mask]
    emotion_preds = emotion_model(np.array(images).squeeze())
    emotion_preds = np.argmax(emotion_preds, axis=1)
    emotion_preds = [label_mapping[Emotion.labels[pred]] for pred in emotion_preds]
    correct_emotions += np.sum(emotion_preds == np.array(labels))
    all_emotion_preds += emotion_preds
    true_emotion_preds += labels
    total += len(labels)

  0%|          | 0/32 [00:00<?, ?it/s]

In [91]:
# Calculate F-score, precision, recall - Race - Training set
precision, recall, f_score_weighted, _ = precision_recall_fscore_support(true_emotion_preds, all_emotion_preds, average='weighted')
_, _, f_score_macro, _ = precision_recall_fscore_support(true_emotion_preds, all_emotion_preds, average='macro')
_, _, f_score_micro, _ = precision_recall_fscore_support(true_emotion_preds, all_emotion_preds, average='micro')
train_acc = np.sum(np.array(all_emotion_preds) == np.array(true_emotion_preds)) / total
print(f"Validation set metrics - Emotion (Deepface) \n" + "="*40)
print(f"Accuracy: {train_acc:.4f} Precision: {precision:.4f}, Recall: {recall:.4f}, F-Score(Weighted): {f_score_weighted:.4f}, F-Score(Micro): {f_score_micro:.4f}, F-Score(Macro): {f_score_macro:.4f}")

Training set metrics - Emotion (Deepface) 
Accuracy: 0.3234 Precision: 0.3418, Recall: 0.3234, F-Score(Weighted): 0.2853, F-Score(Micro): 0.3234, F-Score(Macro): 0.2853
